In [1]:
# D-Wave Leap Authentication
import os
from dwave.cloud import Client

def setup_dwave_config():
    """
    Setup D-Wave configuration for Leap access.
    Make sure you have your API token set up.
    """
    # Option 1: Set environment variable
    # os.environ['DWAVE_API_TOKEN'] = 'your_api_token_here'
    
    # Option 2: Use dwave config file (recommended)
    # Run: dwave config create in terminal to set up interactively
    
    # Test connection
    try:
        with Client.from_config() as client:
            print("✓ Successfully connected to D-Wave Leap")
            return True
    except Exception as e:
        print(f"✗ Connection failed: {e}")
        print("Please set up your D-Wave API token using 'dwave config create'")
        return False

In [2]:
# Test D-Wave connection
setup_dwave_config()

✗ Connection failed: API token not defined
Please set up your D-Wave API token using 'dwave config create'


False

In [3]:
# ==============================================================================
# SETUP AND INSTALLATIONS
# ==============================================================================
print("--- Installing D-Wave libraries... ---")
%pip install dwave-ocean-sdk --upgrade -q

# from google.colab import drive
# import sys, math

# print("\n--- Mounting Google Drive and setting up project path... ---")
# drive.mount('/content/drive', force_remount=True)

# # IMPORTANT: Make sure your project files are in this folder on your Google Drive
# project_path = '/content/drive/MyDrive/qintern-vrp/New-VRP'

# # Add the project path to the system's Python path so it can find your modules
# if project_path not in sys.path:
#     sys.path.append(project_path)
# print(f"Project path '{project_path}' added to system path.")

--- Installing D-Wave libraries... ---
Note: you may need to restart the kernel to use updated packages.


In [3]:
# ==============================================================================
# MAIN EXPERIMENT SCRIPT
# All the logic for loading data, configuring the problem,
# running the solvers, and summarizing the results.
# ==============================================================================
import numpy as np
import time
import math

# Import the refactored modules from your project folder
from input_modified import read_solomon
from vrp_problem import VRPProblem
from vrp_solvers_modified import FullQuboSolver, AveragePartitionSolver

print("\n\n--- Starting VRP Solver Experiment ---")

# ==============================================================================
# EXPERIMENT CONFIGURATION
# ==============================================================================
# please remember to use the full, explicit path to the Solomon data file
SOLOMON_FILE_PATH = 'c101.txt' # Adjust this path as needed
# SOLOMON_FILE_PATH = '/content/drive/MyDrive/qintern-vrp/New-VRP/c101.txt' # Path for Google Drive

NUM_CUSTOMERS = 6
NUM_VEHICLES = 2
LIMIT_RADIUS = 1

# ==============================================================================
# DATA LOADING AND PROBLEM SETUP
# ==============================================================================
print("\n--- Preparing the Vehicle Routing Problem ---")
print(f"Loading data from: {SOLOMON_FILE_PATH}")
print(f"Using the first {NUM_CUSTOMERS} customers.")

problem_data = read_solomon(SOLOMON_FILE_PATH, NUM_CUSTOMERS)
vehicle_capacity = problem_data['capacities'][0]
problem_data['capacities'] = np.array([vehicle_capacity] * NUM_VEHICLES)

problem = VRPProblem(
    sources=problem_data['sources'],
    costs=problem_data['costs'],
    time_costs=problem_data['time_costs'],
    capacities=problem_data['capacities'],
    dests=problem_data['dests'],
    weights=problem_data['weights']
)

print(f"Problem created successfully:")
print(f"  - {len(problem.dests)} destinations (customers)")
print(f"  - {len(problem.capacities)} vehicles")
print(f"  - Vehicle capacity (for post-check): {vehicle_capacity}")

# ==============================================================================
# QUBO FORMULATION PARAMETERS
# ==============================================================================
order_const = 1.0
# The penalty constant must be definitively larger than any possible
# valid route's cost to guarantee valid solutions.
max_edge_cost = np.max(problem.costs)
# A robust penalty is larger than the cost of the most expensive possible tour.
strong_penalty = (NUM_CUSTOMERS + 1) * max_edge_cost * 2.0
only_one_const = strong_penalty
print(f"\nQUBO Parameters: order_const (A1) = {order_const}, only_one_const (A2) = {only_one_const:.2f}")

# ==============================================================================
# EXPERIMENT 1 - FULL QUBO SOLVER (FQS) for VRP
# ==============================================================================
print("\n" + "="*60)
print("--- RUNNING EXPERIMENT: FullQuboSolver (FQS) ---")
print("="*60)

fqs_solver = FullQuboSolver(problem)
print("Solving with classical Simulated Annealing... (this may take a moment)")
start_time_fqs = time.time()
fqs_solution = fqs_solver.solve(only_one_const, order_const, solver_type='simulated')
end_time_fqs = time.time()

print("\n--- FQS RESULTS ---")
print(f"Solver run time: {end_time_fqs - start_time_fqs:.2f} seconds")
fqs_solution.description()
fqs_cost = fqs_solution.total_cost()

# ==============================================================================
# EXPERIMENT 2 - AVERAGE PARTITION SOLVER (APS) for VRP
# ==============================================================================
print("\n" + "="*60)
print("--- RUNNING EXPERIMENT: AveragePartitionSolver (APS) ---")
print("="*60)

aps_solver = AveragePartitionSolver(problem)
print("Solving with classical Simulated Annealing... (this may take a moment)")
start_time_aps = time.time()
aps_solution = aps_solver.solve(only_one_const, order_const, solver_type='simulated', limit_radius=LIMIT_RADIUS)
end_time_aps = time.time()

print("\n--- APS RESULTS ---")
print(f"Solver run time: {end_time_aps - start_time_aps:.2f} seconds")
aps_solution.description()
aps_cost = aps_solution.total_cost()

# ==============================================================================
# SUMMARY
# ==============================================================================
print("\n" + "="*60)
print("--- EXPERIMENT SUMMARY ---")
print("="*60)
print(f"Problem: {NUM_CUSTOMERS} customers, {NUM_VEHICLES} vehicles.")

k_max_fqs = NUM_CUSTOMERS
fqs_limits = [k_max_fqs] * NUM_VEHICLES
fqs_qubo_obj = problem.get_qubo(fqs_limits, only_one_const, order_const)
fqs_num_vars = len(set(v for pair in fqs_qubo_obj.get_dict().keys() for v in pair))

avg_aps = math.ceil(NUM_CUSTOMERS / NUM_VEHICLES)
k_max_aps = avg_aps + LIMIT_RADIUS
aps_limits = [k_max_aps] * NUM_VEHICLES
aps_qubo_obj = problem.get_qubo(aps_limits, only_one_const, order_const)
aps_num_vars = len(set(v for pair in aps_qubo_obj.get_dict().keys() for v in pair))

print("\nQUBO Size Comparison:")
print(f"  - FQS Logical Variables: {fqs_num_vars}")
print(f"  - APS Logical Variables: {aps_num_vars}")
if fqs_num_vars > 0:
    reduction = (fqs_num_vars - aps_num_vars) / fqs_num_vars
    print(f"  - APS achieved a {reduction:.1%} reduction in QUBO variables.")

print("\nSolution Cost Comparison:")
print(f"  - FQS Total Cost: {fqs_cost:.2f}")
print(f"  - APS Total Cost: {aps_cost:.2f}")

print("\nConclusion:")
if aps_cost < fqs_cost:
    print("  As hypothesized in the paper, the APS solver's smaller search space led to a lower-cost solution.")
elif abs(aps_cost - fqs_cost) < 1e-9:
    print("  Both solvers found a solution with the same cost, but APS used fewer variables.")
else:
    print("  In this instance, the FQS solver's larger search space allowed it to find a better solution.")



--- Starting VRP Solver Experiment ---

--- Preparing the Vehicle Routing Problem ---
Loading data from: c101.txt
Using the first 6 customers.
Problem created successfully:
  - 6 destinations (customers)
  - 2 vehicles
  - Vehicle capacity (for post-check): 200

QUBO Parameters: order_const (A1) = 1.0, only_one_const (A2) = 288.62

--- RUNNING EXPERIMENT: FullQuboSolver (FQS) ---
Solving with classical Simulated Annealing... (this may take a moment)

--- FQS RESULTS ---
Solver run time: 0.22 seconds
Solution Routes:
  Vehicle 0: 0 -> 6 -> 3 -> 3 -> 1 -> 0
  Vehicle 1: 0 -> 2 -> 2 -> 5 -> 4 -> 0

Total Cost: 92.45
Is Solution Valid: False

--- RUNNING EXPERIMENT: AveragePartitionSolver (APS) ---
Solving with classical Simulated Annealing... (this may take a moment)

--- APS RESULTS ---
Solver run time: 0.08 seconds
Solution Routes:
  Vehicle 0: 0 -> 3 -> 5 -> 4 -> 0
  Vehicle 1: 0 -> 6 -> 1 -> 2 -> 0

Total Cost: 84.95
Is Solution Valid: True

--- EXPERIMENT SUMMARY ---
Problem: 6 cus

In [ ]:
# Testing all solver types

solver_types = ['simulated', 'exact', 'hybrid']  # Add 'qpu' if you have access

for solver_type in solver_types:
    print(f"\n=== Testing {solver_type} solver ===")
    try:
        # Use a small problem for testing
        test_problem = read_solomon(SOLOMON_FILE_PATH, 3)  # Only 3 customers right now
        # ... run your solver test here
        print(f"✓ {solver_type} solver working correctly")
    except Exception as e:
        print(f"✗ {solver_type} solver failed: {e}")


=== Testing simulated solver ===
✓ simulated solver working correctly

=== Testing exact solver ===
✓ exact solver working correctly

=== Testing hybrid solver ===
✓ hybrid solver working correctly
